In [1]:
import pandas as pd
pd.set_option("future.no_silent_downcasting", True)

from wsi.indicators.education import build_education_df
from wsi.indicators.employment import build_employment_df
from wsi.indicators.parliament import build_parliamentary_df
from wsi.indicators.poverty import build_poverty_df
from wsi.indicators.legal import build_legal_df
from wsi.indicators.son_bias import build_son_bias_df
from wsi.indicators.maternal_mortality import build_maternal_mortality_df
from wsi.indicators.attitude_violence import build_attitudes_violence_df
from wsi.indicators.child_marriage import build_child_marriage_df
from wsi.indicators.water_sanitation import build_water_sanitation_df
from wsi.indicators.electricity import build_access_electricity_df
from wsi.indicators.financial_inclusion import build_financial_inclusion_df



In [3]:
build_financial_inclusion_df()

,ISO_code,Year,Financial Inclusion
0,AFG,2011,0.026162
1,AFG,2014,0.038124
2,AFG,2017,0.066383
3,AFG,2021,0.046973
4,ALB,2011,0.226726
...,...,...,...
653,SSA,2021,0.337484
654,WLD,2011,0.466166
655,WLD,2014,0.576834
656,WLD,2017,0.636905


In [3]:
build_access_electricity_df()

,ISO_code,Year,Access Electricity
0,ALB,1990,100.000000
1,AND,1990,100.000000
2,ATG,1990,95.763046
3,ARG,1990,92.154800
4,ABW,1990,100.000000
...,...,...,...
7608,TSA,2022,98.287996
7609,SSF,2022,51.434739
7610,SSA,2022,51.429932
7611,TSS,2022,51.434739


In [2]:
a = build_water_sanitation_df()
a

,ISO_code,Year,Access Water Sanitation
0,ABW,2000,96.823286
1,ABW,2001,96.942643
2,ABW,2002,97.062000
3,ABW,2003,97.181356
4,ABW,2004,97.300713
...,...,...,...
5938,ZWE,2018,45.327092
5939,ZWE,2019,44.790140
5940,ZWE,2020,44.258813
5941,ZWE,2021,43.732090


In [4]:
build_child_marriage_df()

,ISO_code,Year,Child Marriage
0,SLV,1985,37.7
1,BRA,1986,21.8
2,COL,1986,23.3
3,DOM,1986,34.4
4,LBR,1986,48.4
...,...,...,...
411,UZB,2022,3.4
412,AFG,2023,28.7
413,MOZ,2023,48.4
414,SGP,2023,0.1


In [ ]:
build_attitudes_violence_df()

In [ ]:
build_maternal_mortality_df()

In [ ]:
a = build_son_bias_df()

a['Year'].max()

In [ ]:
df= build_legal_df()
df


In [ ]:
build_poverty_df()

In [ ]:
build_parliamentary_df()

In [ ]:
edu_world = build_education_df()
edu_world['Year'].max()

In [ ]:
emp_world = build_employment_df()
emp_world

In [ ]:
emp_world['Year'].max()